<a href="https://colab.research.google.com/github/Pbrillan/CEIA/blob/main/NLP/Pbrillan_3b_Custom_embedding_con_Gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [2]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['blink-182.txt',
 'kanye.txt',
 'adele.txt',
 'Kanye_West.txt',
 'r-kelly.txt',
 'eminem.txt',
 'kanye-west.txt',
 'lady-gaga.txt',
 'lin-manuel-miranda.txt',
 'leonard-cohen.txt',
 'lorde.txt',
 'missy-elliott.txt',
 'notorious-big.txt',
 'prince.txt',
 'radiohead.txt',
 'lil-wayne.txt',
 'drake.txt',
 'disney.txt',
 'amy-winehouse.txt',
 'bieber.txt',
 'nicki-minaj.txt',
 'notorious_big.txt',
 'al-green.txt',
 'bruno-mars.txt',
 'nickelback.txt',
 'dj-khaled.txt',
 'dolly-parton.txt',
 'michael-jackson.txt',
 'britney-spears.txt',
 'bob-marley.txt',
 'Lil_Wayne.txt',
 'ludacris.txt',
 'dr-seuss.txt',
 'beatles.txt',
 'rihanna.txt',
 'nursery_rhymes.txt',
 'joni-mitchell.txt',
 'janisjoplin.txt',
 'nirvana.txt',
 'bruce-springsteen.txt',
 'alicia-keys.txt',
 'paul-simon.txt',
 'patti-smith.txt',
 'jimi-hendrix.txt',
 'bjork.txt',
 'dickinson.txt',
 'cake.txt',
 'bob-dylan.txt',
 'johnny-cash.txt']

In [4]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/bob-marley.txt', sep='/n', header=None)
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0
0,"""Don't worry about a thing,"
1,'Cause every little thing gonna be all right.
2,"Singin': ""Don't worry about a thing,"
3,"'Cause every little thing gonna be all right!""..."
4,"Smiled with the risin' sun,"


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2218


### 1 - Preprocesamiento

In [6]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
# Demos un vistazo
sentence_tokens[:2]

[["don't", 'worry', 'about', 'a', 'thing'],
 ["'cause", 'every', 'little', 'thing', 'gonna', 'be', 'all', 'right']]

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2218


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 532


### 3 - Entrenar el modelo generador

In [13]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 153434.921875
Loss after epoch 1: 93280.90625
Loss after epoch 2: 92533.015625
Loss after epoch 3: 93047.125
Loss after epoch 4: 93667.90625
Loss after epoch 5: 92701.25
Loss after epoch 6: 87062.625
Loss after epoch 7: 83619.5625
Loss after epoch 8: 79414.125
Loss after epoch 9: 75504.6875
Loss after epoch 10: 73822.6875
Loss after epoch 11: 68258.8125
Loss after epoch 12: 64552.125
Loss after epoch 13: 62677.625
Loss after epoch 14: 62621.625
Loss after epoch 15: 62262.125
Loss after epoch 16: 61252.75
Loss after epoch 17: 61218.5
Loss after epoch 18: 60727.0
Loss after epoch 19: 59957.25


(218093, 359260)

### 4 - Ensayar

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["cry"], topn=10)

[('woman', 0.987288236618042),
 ('no', 0.9427759647369385),
 ('fowl', 0.9262357354164124),
 ('tears', 0.9179865717887878),
 ('call', 0.8933768272399902),
 ('matter', 0.8843841552734375),
 ("ain't", 0.8743000030517578),
 ('nice', 0.8629266619682312),
 ('hypocrites', 0.8561786413192749),
 ('shed', 0.8492369651794434)]

In [15]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["cry"], topn=10)

[('doo', -0.2110741138458252),
 ('fire', -0.22161422669887543),
 ('with', -0.22227106988430023),
 ('music', -0.22687682509422302),
 ('youths', -0.22832322120666504),
 ('like', -0.23266032338142395),
 ('this', -0.23503893613815308),
 ('think', -0.23944133520126343),
 ('oo', -0.24549664556980133),
 ('jamaica', -0.24604079127311707)]

In [16]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["natty"], topn=10)

[('ride', 0.9863688349723816),
 ('rides', 0.9803694486618042),
 ('dread', 0.9555577039718628),
 ('again', 0.9428709149360657),
 ('walk', 0.9181586503982544),
 ('once', 0.846028745174408),
 ('roots', 0.8155065774917603),
 ('hey', 0.8051044940948486),
 ('whoa', 0.8041038513183594),
 ('ah', 0.7922512292861938)]

In [17]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["natty"], topn=10)

[('like', -0.23835371434688568),
 ('youths', -0.25346750020980835),
 ('this', -0.2558543086051941),
 ('was', -0.2594929039478302),
 ('great', -0.26547399163246155),
 ('with', -0.2693966329097748),
 ('day', -0.2735257148742676),
 ('love', -0.27379927039146423),
 ('he', -0.27867987751960754),
 ('music', -0.28378981351852417)]

In [18]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["every"], topn=5)

[('night', 0.8944805264472961),
 ('day', 0.8883407115936279),
 ("'cause", 0.875667154788971),
 ('three', 0.8670030236244202),
 ('birds', 0.8412587642669678)]

In [19]:
# Ensayar con una palabra que no está en el corpus (en vocab):
w2v_model.wv.most_similar(negative=["hola"])

KeyError: ignored

### 5 - Visualizar agrupación de vectores

In [20]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [21]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
